# Trabajo con dataframes para reto "Crédito PyMe" - Hackaton BBVA

In [1]:
import pandas as pd
from datetime import date
import numpy as np

In [2]:
# Leemos los archivos csv
df_Ranking = pd.read_csv(r"dbRanking.csv", sep="|", encoding='utf8')
df_Rcc = pd.read_csv(r"dbRcc.csv", sep="|", encoding='utf8')
df_Universo = pd.read_csv(r"dbUniverso.csv", sep="|", encoding='utf8')


## Tratamiento inicial de dataframes y primer merge entre Rcc y Universo

In [3]:
# Eliminamos las columnas
df_Rcc.drop(columns = ["sbs_entity_id", "cutoff_date"], inplace = True)
df_Universo.drop(columns = ["personal_type", "gender_type", "birth_date", "marital_status_type", "income_monthly_amount", "cutoff_date"], inplace = True)

In [4]:
# Convertir df_Universo["sbs_customer_id"] a dtype int64 para evitar la notación científica
df_Universo["sbs_customer_id"] = df_Universo["sbs_customer_id"].convert_dtypes("int64")

In [5]:
# Quedamos con los registros de df_Universo que tienen un valor de sbs_customer_id
df_temp = df_Universo[df_Universo["sbs_customer_id"].isin(df_Universo["sbs_customer_id"].dropna())]
df_temp = pd.DataFrame(df_temp)

In [6]:
# Unimos las tablas Universo y Rcc
df_total = pd.merge(left= df_temp, right= df_Rcc)

## Limpiar el df_Ranking

In [7]:
# Columnas de Ranking que no son el último valor registrado de año: 133-157, 159-183, 184-198
columnas_borradas = list(range(133,157)) + list(range(159,183)) + list(range(184,198)) + list(range(216, 225))
df_Ranking.drop(columns = df_Ranking.columns[columnas_borradas], inplace = True)

In [8]:
# Armamos un vector con los índices de las columnas con menos de la mitad de valores vacíos
guardar = []
for col in range(7, len(df_Ranking.columns)):
    vacios = sum(df_Ranking.iloc[:, col].isna())
    if vacios < len(df_Ranking)/2:
        guardar.append(col) 
        print(f"Col {col}, {df_Ranking.columns[col]}: {vacios}")

Col 7, seniority_company_years_number: 320
Col 8, company_size_desc: 34
Col 10, department_name: 2
Col 11, province_name: 0
Col 12, district_name: 0
Col 13, private_public_mark_desc: 0
Col 14, market_share_per: 274
Col 132, usd_export_2017_amount: 0
Col 133, importer_mark_type: 6547
Col 134, usd_import_2017_amount: 0
Col 135, state_sales_2016_amount: 0
Col 153, pbi_sector_2016_per: 9
Col 154, cutoff_date: 0


In [9]:
# Incluimos las primeras 7 columnas
guardar = list(range(7)) + guardar

# Recreamos el dataframe con las columnas seleccionadas
df_Ranking = pd.DataFrame(df_Ranking.iloc[:,guardar])

In [10]:
# Borramos las columnas con datos que no usaremos
df_Ranking.drop(columns= ["company_economic_activity_id",
                          "ciiu_spanish_desc",
                          "private_public_mark_desc",
                          "activity_start_date",
                          #"province_name",
                          "district_name",
                          "importer_mark_type",
                          "cutoff_date"], inplace = True)

## Merge del df_total y  df_Ranking

In [11]:
# Merge del dataframe y nuestra tabla Ranking trabajada
df_total = pd.merge(left= df_total, right= df_Ranking)

# Borramos las columnas con IDs
# df_total.drop(columns = ["customer_id", "sbs_customer_id", "taxpayer_id"], inplace = True)

# Seleccionamos las Pequeñas y Medianas empresas
df_total = pd.DataFrame(df_total[ df_total["company_size_desc"] != "Gran Empresa / Big-size Company" ], index=None)


In [12]:
df_total.reset_index(inplace = True)
df_total.drop(columns = ["index"], inplace = True)

In [13]:
# Borramos todas las filas con valores vacíos (NaN)
filas_para_borrar = []
for i in df_total.index:
    if sum(df_total.iloc[i].isna()) > 0:
        filas_para_borrar.append(i)

df_total.drop(index=filas_para_borrar, inplace = True)

## Tratamiento de variables

In [14]:
# Variable "admission_date"

año_actual = date.today().year
temporal = pd.to_datetime(df_total["admission_date"], format= "%Y-%m-%d")

# Reemplazamos los valores de admission_date por los años respecto al año atual
df_total["admission_date"] = pd.Series ( [año_actual - temporal.iloc[x].year for x in range(len(temporal))] )

# Eliminamos las filas con valores NaN
df_total = pd.DataFrame(df_total[ [not x for x in df_total["admission_date"].isnull()] ])

In [15]:
# Variable "seniority_company_years_number"

def año_funcionamiento(x):
    return x + 2
df_total["año_funcionamiento"] = df_total.apply(lambda x: año_funcionamiento(x.seniority_company_years_number), axis=1)

# Borrar últimas variables
df_total.drop(columns = ["seniority_company_years_number", "segmento", "spanish_sector_company_name"], inplace = True)

In [16]:
df_total.reset_index(inplace = True)
df_total.drop(columns = ["index"], inplace = True)

In [17]:
# Variable con localización

# Importamos datos de ingreso promedio para cada una de las provincias del país
df_ubigeos = pd.read_csv("ProvinciaEconomico.csv")

In [18]:
# Corregir nombres de provincias
for fila in df_total[df_total["province_name"] == 'HuÃ¡nuco'].index:
    df_total.loc[fila, "province_name"] = 'Huánuco'

for fila in df_total[df_total["province_name"] == 'San RomÃ¡n'].index:
    df_total.loc[fila, "province_name"] = 'San Román'

for fila in df_total[df_total["province_name"] == 'San MartÃ­n'].index:
    df_total.loc[fila, "province_name"] = 'San Martín'

for fila in df_total[df_total["province_name"] == 'JaÃ©n'].index:
    df_total.loc[fila, "province_name"] = 'Jaén'

for fila in df_total[df_total["province_name"] == 'La ConvenciÃ³n'].index:
    df_total.loc[fila, "province_name"] = 'La Convención'

for fila in df_total[df_total["province_name"] == 'CaÃ±ete'].index:
    df_total.loc[fila, "province_name"] = 'Cañete'

for fila in df_total[df_total["province_name"] == 'BongarÃ¡'].index:
    df_total.loc[fila, "province_name"] = 'Bongará'

for fila in df_total[df_total["province_name"] == 'CamanÃ¡'].index:
    df_total.loc[fila, "province_name"] = 'Camaná'


In [19]:
# Corregir nombres de departamentos
for fila in df_total[df_total["department_name"] == 'JunÃ­n'].index:
    df_total.loc[fila, "department_name"] = 'Juní­n'

for fila in df_total[df_total["department_name"] == 'HuÃ¡nuco'].index:
    df_total.loc[fila, "department_name"] = 'Huánuco'

for fila in df_total[df_total["department_name"] == 'San MartÃ­n'].index:
    df_total.loc[fila, "department_name"] = 'San Martín'

for fila in df_total[df_total["department_name"] == 'ApurÃ­mac'].index:
    df_total.loc[fila, "department_name"] = 'Apurímac'

for fila in df_total[df_total["department_name"] == 'Callao'].index:
    df_total.loc[fila, "department_name"] = 'Lima'


In [20]:
#Agregamos una variable de ingresos promedio en vez de las variables de provincia y departamento
ingreso_promedio_provincia = []
departamento_temp = []
provincia_temp = []
for i in df_total.index:
    for j in df_ubigeos.index:
        if (df_total.loc[i, "province_name"]==df_ubigeos.loc[j, "PROVINCIA"]):
            ingreso_promedio_provincia.append(df_ubigeos.loc[j, 'P_Ingresos'])
df_total["ingreso_promedio_provincia"] = pd.Series(ingreso_promedio_provincia)
df_total.drop(columns = ["department_name", "province_name"], inplace = True)

## Armar variable de salida

In [21]:
df_total.reset_index(inplace = True)
df_total.drop(columns = ["index"], inplace = True)

In [22]:
def umbralBinario(I,u1):
    out = np.where((I <= u1),1,0)                
    return out

new = []
for i in range(len(df_total["num_days_default_payment_number"])):
    new.append(umbralBinario(df_total.loc[i, "num_days_default_payment_number"],60))
    
X_bin = np.array(new)
temp = pd.Series(X_bin)
df_total["num_days_default_payment_number"] = temp

## Borramos las variables categóricas producto, subproducto, situación de credito, company_size_desc y nombre de provincia/departamento

In [23]:
df_pequeñas = df_total[df_total["company_size_desc"] == "PequeÃ±a empresa / Small-size Company"]

In [24]:
df_pequeñas["num_days_default_payment_number"].value_counts()

1    19318
0     1397
Name: num_days_default_payment_number, dtype: int64

In [25]:
# Borramos las variables categóricas
df_total.drop(columns = ["producto", "subproducto", "situacion_credito", "company_size_desc"], inplace = True)

In [26]:
# Mandar la variable de salida al final del dataframe
columnas = list(df_total.columns)
columnas.remove("num_days_default_payment_number")
columnas.append("num_days_default_payment_number")

df_total = df_total.reindex(columns = columnas)

In [27]:
df_total

,personal_id,customer_id,sbs_customer_id,taxpayer_id,admission_date,balance_amount,employees_number,branches_number,market_share_per,usd_export_2017_amount,usd_import_2017_amount,state_sales_2016_amount,pbi_sector_2016_per,año_funcionamiento,ingreso_promedio_provincia,num_days_default_payment_number
0,52612482530,11166569,5753661293,52612482530,19.0,85108.30,64.0,1.0,0.282457,601786.28,2760.0,0.0,2.5,22.0,40100,1
1,52612482530,11166569,5753661293,52612482530,19.0,85108.30,64.0,1.0,0.282457,601786.28,2760.0,0.0,2.5,22.0,40100,1
2,52612482530,11166569,5753661293,52612482530,19.0,180271.95,64.0,1.0,0.282457,601786.28,2760.0,0.0,2.5,22.0,40100,1
3,52612482530,11166569,5753661293,52612482530,19.0,180271.95,64.0,1.0,0.282457,601786.28,2760.0,0.0,2.5,22.0,40100,1
4,52612482530,11166569,5753661293,52612482530,19.0,302930.16,64.0,1.0,0.282457,601786.28,2760.0,0.0,2.5,22.0,40100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88404,6520110151,46018402,9569563909,6520110151,4.0,789.83,39.0,4.0,0.175037,0.00,0.0,0.0,2.0,21.0,56358,1
88405,6520110151,46018402,9569563909,6520110151,4.0,789.83,39.0,4.0,0.175037,0.00,0.0,0.0,2.0,21.0,56358,1
88406,6520110151,46018402,9569563909,6520110151,4.0,20833.30,39.0,4.0,0.175037,0.00,0.0,0.0,2.0,21.0,56358,1
88407,6520110151,46018402,9569563909,6520110151,4.0,20833.30,39.0,4.0,0.175037,0.00,0.0,0.0,2.0,21.0,56358,1


In [28]:
df_total.to_csv("dataset_credito_pyme.csv", encoding="utf-8", index=False)